# 로지스틱 회귀
- 선형회귀에서 출발
- 이진 분류 문제 해결하기위해 사용
- 클래스 확률 예측
- 딥러닝에서도 사용

## 117p
- x가 사각형일 확률 30%
- x가 삼각형일 확률 50%
- x가 원일 확률 20% 

## 데이터 불러오기
- 종속변수(Y) = Species
- 독립변수(X) = Weight, Length, Diagonal, Height, Width

In [14]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
print(fish.head())

  Species  Weight  Length  Diagonal   Height   Width
0   Bream   242.0    25.4      30.0  11.5200  4.0200
1   Bream   290.0    26.3      31.2  12.4800  4.3056
2   Bream   340.0    26.5      31.1  12.3778  4.6961
3   Bream   363.0    29.0      33.5  12.7300  4.4555
4   Bream   430.0    29.0      34.0  12.4440  5.1340


## 데이터 탐색
- 종속변수부터 확인

In [15]:
print(pd.unique(fish['Species']))
# print(fish['Species'].value_counts())

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


## 데이터 가공

In [16]:
# pandas 데이터프레임에서 numpy 배열로 변환
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_input.shape

(159, 5)

In [17]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


- 종속변수, 타겟데이터, y도 2차원배열로 

In [18]:
fish_target = fish['Species'].to_numpy()
print(fish_target.shape)
print(fish_target[:5]) 

(159,)
['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


## 데이터 분리
- 훈련데이터 테스트 데이터 분리

In [30]:
from sklearn.model_selection import train_test_split
# 임의 샘플링
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)
# 층화 샘플링 - 비율맞추기

## 데이터 전처리

### 표준화 
- 훈련 세트의 통계값으로 테스트 세트를 변환해야 한다
- 데이터 가공
  - 결측치를 훈련세트의 평균값으로 대체를 하는데
테스트세트의 결측치는 테스트 세트의 평균값을 모르니  결측치를 대체하지 못한다.
  - 그럴때는 훈련세트의 평균값을 넣어야한다.
  - 원본데이터의 평균X
  - 테스트세트의 평균과 훈련세트의 평균이 달라도 훈련세트의 평균을 넣어줘야 한다.
- 데이터 누수(Data Leakage)
 - 훈련데이터 평균값 대치  (o)
 - 테스트데이터 평균값 대치(x)
 - 모든 데이터 평균값 대치 (x)

In [31]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input) # 훈련데이터
#ss.fit(test_input)(X) <- 테스트데이터

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

## 모형 만들기 
- k 최근접이웃 

In [32]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled,train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [34]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


### 다중분류

In [35]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


### 5개 샘플에 대한 예측은 어떤 확률이냐 
- proba

In [38]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(kn.classes_)
print(np.round(proba,decimals=4))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


- 첫번째 클래스 : Perch
  - 100%확률로 Perch 예측
- 네번째 클래스 : Perch
  - 66.7% 확률로 Perch 예측
  - 33.3% 확률로 Roach로 예측

## 회귀 식
- $ y = ax + b$
- (y = 연속형 데이터)일 때 사용 


## 로지스틱 회귀로 이진분류 

In [42]:
char_arr = np.array(['A', 'B', 'C', 'D','E'])
print(char_arr[[True, False, True, False, False]]) # True값만 추출

['A' 'C']


- 도미와 빙어의 행만 골라냄 

In [54]:
bream_smelt_indexes = ( train_target == 'Bream')|(train_target =="Smelt") # True,False로 추출
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

train_scaled.shape, train_bream_smelt.shape

((119, 5), (33, 5))

## 모델만들기 

In [57]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [61]:
# 클래스를 예측
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


- 확률을 확인

In [62]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [64]:
print(lr.classes_) # 클래스 속성확인 

['Bream' 'Smelt']


- 분류기준 : threshold 임계값설정

### 계수와 절편
- $z = -0.404*(Weight) - 0.576*(Length) -0.663*(Diagonal) - 1.013*(Height) - 0.732*(Width) - 2.162 $

In [67]:
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


- z값

In [69]:
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


- 우리가 원한건 확률이라 확률로 변환

In [70]:
from scipy.special import expit
print(expit(decisions)) # 확률값으로 변환

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


## 디중분류 수행
- 이중분류의 확장판 

In [73]:
# 하이퍼 파라미터 세팅
# 모형을 튜닝
# 모형 결과의 과대적합|과소적합을 방지하기 위한것
lr = LogisticRegression(C = 20, max_iter = 1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [77]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [78]:
# 확률값
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


- 다중 분류일 경우 선형 방정식
  - 7개의 분류값, 컬럼값 5개에 대한 선형방정식이 나온다 

In [80]:
print(lr.coef_.shape,lr.intercept_.shape) # 각각의 기울기와 상수값

(7, 5) (7,)


# 평가지표 

## 회귀 평가지표
- 결정계수
  - $(타깃-예측)^2의합 / (타깃-평균)^2의 합$
- MAE, MSE, RMSE
  - (실제 - 예측) = 오차
  - MAE(Mean Absolute Error) = |오차|(오차의 절댓값)의 평균
  - MSE(Mean Squared Error) = 오차^2의 평균
  - RMSE(Root Mean Squared Error) = MSE의 제곱근
- 좋은모델
  - 결정계수가 1에 수렴할수록
  - MAE, MSE, RMSE는 0에 수렵할수록 

In [89]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

true = np.array([1, 2, 3, 2, 3, 5, 4, 6, 5, 6, 7, 8, 8]) # 실제값
preds = np.array([1, 1, 2, 2, 3, 4, 4, 5, 5, 7, 7, 6, 8]) # 예측값

# 절대값 오차의 평균
mae = mean_absolute_error(true, preds)
print("---mae---")
print(mae)
print("---mse---")

# 제곱 오차의 평균
mse = mean_squared_error(true, preds)
print(mse)
print("---rmse---")

# mse의 제곱근
rmse = np.sqrt(mse)
print(rmse)
print("---결정계수---")

#결정계수
r2 = r2_score(true, preds)
print(r2)

---mae---
0.5384615384615384
---mse---
0.6923076923076923
---rmse---
0.8320502943378437
---결정계수---
0.8617021276595744


## 분류 평가지표


### 오차 행렬(Confusion Matrix)
- ex)
  - 실제 값
    - [빙어, 도미, 도미, 빙어, 도미]
  - 예측 값
    - [빙어, 빙어, 도미, 빙어, 빙어]
  - TP(빙어를 빙어로 예측) : 2
  - TN(도미를 도미로 예측) : 1
  - FN(실제=도미, 예측=빙어) : 2
  - FP(실제=빙어, 예측=도미) : 0
- TP,TN,FP,FN
  - 정확도
    - (TP + TN) / (TP + TN + FP + FN)
  - 정밀도(Precision) : 양성이라고 예측한 값(TP+FP) 중 실제 양성(TP)의 비율
    - TP / (TP + FP)
  - 재현율 : 실제 양성값(TP+FN) 중 양성으로 예측한 값(TP)
    - TP / (TP + FN)
  - 로그손실
  - ROC Curve(=AUC)

- 상황에 따른 정확도, 정밀도, 재현율을 구분을 잘하고 우선순위를 따져야한다.

In [90]:
from sklearn.metrics import confusion_matrix

true = [0,1,1,0,0]
preds = [1,0,0,0,0]

confusion_matrix(true, preds)

array([[2, 1],
       [2, 0]])